# Beat Analyzer

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import wave, sys

# Change this to the desired file name
raw = wave.open("music.wav")

signal = raw.readframes(-1)
signal = np.frombuffer(signal, dtype ="int16")

f_rate = raw.getframerate()

time = np.linspace(0, len(signal)/f_rate, num = len(signal))

plt.title("Sound Wave")
plt.xlabel("Time")
plt.plot(time[:int(5e5)], signal[:int(5e5)])
plt.show()

## Spectogram

Using matplotlib's [`specgram`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.specgram.html#matplotlib.pyplot.specgram) function, we can find the frequencies within times. 

In [ ]:
ranges = np.array([0, -1], dtype=int)
print(f"Time range = {time[ranges[0]]} - {time[ranges[1]]}")

result = plt.specgram(signal[ranges[0]:ranges[1]], NFFT=256, Fs=f_rate, cmap="jet")
plt.colorbar()
plt.show()

In [ ]:
spectrum = result[0]
freqs = result[1]
times = result[2]

# Sorts the spectrum into categories
bass = spectrum[np.where((20 < freqs) & (freqs < 250))]
midrange = spectrum[np.where((250 < freqs) & (freqs < 4000))]
high = spectrum[freqs > 4000]

# If you want custom frequencies that you think the beats are in
custom_freqs = [250, 4000]
custom = spectrum[np.where((custom_freqs[0] < freqs) & (freqs < custom_freqs[1]))]

# The time resolution from the spectogram
dt = times[1] - times[0]

# Sum of the frequencies' magnitudes 
magnitudes = np.sum(spectrum, axis=0) # axis = 0, column addition
bass_mag = np.sum(bass, axis=0)
midrange_mag = np.sum(midrange, axis=0)
high_mag = np.sum(high, axis=0)
custom_mag = np.sum(custom, axis=0)

# Uncomment to see the magnitudes plot
# plt.plot(times + time[ranges[0]], magnitudes)
# plt.plot(times + time[ranges[0]], bass_mag)
plt.plot(times + time[ranges[0]], midrange_mag)
# plt.plot(times + time[ranges[0]], high_mag)
# plt.plot(times + time[ranges[0]], custom_mag)

plt.xlabel("Time (s)")
plt.ylabel("Magnitude")
plt.show()

## Find Beat Times

In [ ]:
from scipy.signal import find_peaks

# https://stackoverflow.com/questions/14313510/how-to-calculate-rolling-moving-average-using-python-numpy-scipy
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

# Moving average to smooth out the curves and normalize it
# Change this to either bass_mag, midrange_mag, high_mag, magnitudes, or custom_mag for desired peaks
magnitudes_rolling = moving_average(midrange_mag, 30)
magnitudes_rolling_norm = magnitudes_rolling/np.max(magnitudes_rolling)

# Find the peaks where the distance between the peaks are more than 0.3 secs away
peaks = find_peaks(magnitudes_rolling_norm, distance=0.3/(dt))

plt.plot((result[2] + time[ranges[0]])[:len(magnitudes_rolling_norm)], magnitudes_rolling_norm)
plt.scatter(result[2][peaks[0]] + time[ranges[0]], magnitudes_rolling_norm[peaks[0]], color="tab:orange")
plt.show()

In [ ]:
peak_times = result[2][peaks[0]] + time[ranges[0]]
print("Peak times in seconds")
print(peak_times.tolist())

## References

1. [Tempo and Beat Tracking, www.youtube.com](https://www.youtube.com/watch?v=FmwpkdcAXl0&t=576s)